In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Documents/programming/BME-574-2024`


In [3]:
Pkg.add(["Lux", "ADTypes", "MLUtils", "Optimisers", "Zygote", "OneHotArrays", "Random", "Statistics", "Printf", "MLDatasets"])

    Updating registry at `~/.julia/registries/JSMLComponents.toml`
    Updating registry at `~/.julia/registries/JuliaComputingRegistry.toml`
    Updating registry at `~/.julia/registries/JuliaHubRegistry.toml`
   Resolving package versions...
  No Changes to `~/Documents/programming/BME-574-2024/Project.toml`
  No Changes to `~/Documents/programming/BME-574-2024/Manifest.toml`
Precompiling project...
  ✓ XSLT_jll
  ✓ SparseMatrixColorings → SparseMatrixColoringsColorsExt
  ✓ LsqFit
  ✓ SteadyStateDiffEq
  ✓ Plots → UnitfulExt
  ✓ LinearSolve → LinearSolveKernelAbstractionsExt
  ✓ RootedTrees → PlotsExt
  ✓ NonlinearSolve → NonlinearSolveNLsolveExt
  ✓ Sundials
  ✓ StochasticDiffEq
  10 dependencies successfully precompiled in 13 seconds. 751 already precompiled. 2 skipped during auto due to previous errors.


In [6]:
using Lux, ADTypes, MLUtils, Optimisers, Zygote, OneHotArrays, Random, Statistics, Printf
using MLDatasets: MNIST

In [7]:
function loadmnist(batchsize, train_split)
    # Load MNIST
    N = 60000
    dataset = MNIST(; split=:train)
    imgs = dataset.features[:, :, 1:N]
    labels_raw = dataset.targets[1:N]

    # Process images into (H, W, C, BS) batches
    x_data = Float32.(reshape(imgs, size(imgs, 1), size(imgs, 2), 1, size(imgs, 3)))
    y_data = onehotbatch(labels_raw, 0:9)
    (x_train, y_train), (x_test, y_test) = splitobs((x_data, y_data); at=train_split)

    return (
        # Use DataLoader to automatically minibatch and shuffle the data
        DataLoader(collect.((x_train, y_train)); batchsize, shuffle=true),
        # Don't shuffle the test data
        DataLoader(collect.((x_test, y_test)); batchsize, shuffle=false))
end

loadmnist (generic function with 1 method)

In [8]:
lux_model = Chain(Conv((5, 5), 1 => 6, relu), MaxPool((2, 2)),
    Conv((5, 5), 6 => 16, relu), MaxPool((2, 2)), FlattenLayer(3),
    Chain(Dense(256 => 128, relu), Dense(128 => 84, relu),Dense(84 => 10)))

Chain(
    layer_1 = Conv((5, 5), 1 => 6, relu),  # 156 parameters
    layer_2 = MaxPool((2, 2)),
    layer_3 = Conv((5, 5), 6 => 16, relu),  # 2_416 parameters
    layer_4 = MaxPool((2, 2)),
    layer_5 = FlattenLayer{Static.StaticInt{3}}(static(3)),
    layer_6 = Chain(
        layer_1 = Dense(256 => 128, relu),  # 32_896 parameters
        layer_2 = Dense(128 => 84, relu),  # 10_836 parameters
        layer_3 = Dense(84 => 10),      # 850 parameters
    ),
)         # Total: 47_154 parameters,
          #        plus 0 states.

In [32]:
lux_model2 = Chain(Conv((3, 3), 1 => 32, relu), Conv((3, 3), 32 => 64, relu), MaxPool((3, 3)),
    Dropout(0.5), FlattenLayer(3),
    Chain(Dense( 4096 => 250, relu), Dense(250 => 10)))

Chain(
    layer_1 = Conv((3, 3), 1 => 32, relu),  # 320 parameters
    layer_2 = Conv((3, 3), 32 => 64, relu),  # 18_496 parameters
    layer_3 = MaxPool((3, 3)),
    layer_4 = Dropout(0.5),
    layer_5 = FlattenLayer{Static.StaticInt{3}}(static(3)),
    layer_6 = Chain(
        layer_1 = Dense(4096 => 250, relu),  # 1_024_250 parameters
        layer_2 = Dense(250 => 10),     # 2_510 parameters
    ),
)         # Total: 1_045_576 parameters,
          #        plus 2 states.

In [9]:
const loss = CrossEntropyLoss(; logits=Val(true))

function accuracy(model, ps, st, dataloader)
    total_correct, total = 0, 0
    st = Lux.testmode(st)
    for (x, y) in dataloader
        target_class = onecold(y)
        predicted_class = onecold(Array(first(model(x, ps, st))))
        total_correct += sum(target_class .== predicted_class)
        total += length(target_class)
    end
    return total_correct / total
end

accuracy (generic function with 1 method)

In [10]:
function train(model; rng=Xoshiro(0), kwargs...)
    train_dataloader, test_dataloader = loadmnist(128, 0.9)
    ps, st = Lux.setup(rng, model)

    train_state = Training.TrainState(model, ps, st, Adam(3.0f-4))

    ### Warmup the model
    x_proto = randn(rng, Float32, 28, 28, 1, 1)
    y_proto = onehotbatch([1], 0:9)
    Training.compute_gradients(AutoZygote(), loss, (x_proto, y_proto), train_state)

    ### Lets train the model
    nepochs = 10
    tr_acc, te_acc = 0.0, 0.0
    for epoch in 1:nepochs
        stime = time()
        for (x, y) in train_dataloader
            gs, _, _, train_state = Training.single_train_step!(
                AutoZygote(), loss, (x, y), train_state)
        end
        ttime = time() - stime

        tr_acc = accuracy(
            model, train_state.parameters, train_state.states, train_dataloader) * 100
        te_acc = accuracy(
            model, train_state.parameters, train_state.states, test_dataloader) * 100

        @printf "[%2d/%2d] \t Time %.2fs \t Training Accuracy: %.2f%% \t Test Accuracy: \
                 %.2f%%\n" epoch nepochs ttime tr_acc te_acc
    end

    return tr_acc, te_acc
end

train (generic function with 1 method)

In [11]:
tr_acc, te_acc = train(lux_model)

[ 1/10] 	 Time 8.30s 	 Training Accuracy: 92.06% 	 Test Accuracy: 93.95%
[ 2/10] 	 Time 5.44s 	 Training Accuracy: 95.21% 	 Test Accuracy: 95.67%
[ 3/10] 	 Time 5.59s 	 Training Accuracy: 96.50% 	 Test Accuracy: 96.68%
[ 4/10] 	 Time 5.62s 	 Training Accuracy: 97.37% 	 Test Accuracy: 97.38%
[ 5/10] 	 Time 5.55s 	 Training Accuracy: 97.82% 	 Test Accuracy: 97.48%
[ 6/10] 	 Time 5.68s 	 Training Accuracy: 97.91% 	 Test Accuracy: 97.57%
[ 7/10] 	 Time 5.56s 	 Training Accuracy: 98.01% 	 Test Accuracy: 97.63%
[ 8/10] 	 Time 5.50s 	 Training Accuracy: 98.33% 	 Test Accuracy: 97.88%
[ 9/10] 	 Time 5.58s 	 Training Accuracy: 98.62% 	 Test Accuracy: 98.13%
[10/10] 	 Time 5.52s 	 Training Accuracy: 98.64% 	 Test Accuracy: 97.95%


(98.63518518518518, 97.95)

In [52]:
tr_acc2, te_acc2 = train(lux_model2)

[ 1/10] 	 Time 74.42s 	 Training Accuracy: 97.84% 	 Test Accuracy: 98.37%
[ 2/10] 	 Time 71.04s 	 Training Accuracy: 98.69% 	 Test Accuracy: 98.72%
[ 3/10] 	 Time 71.29s 	 Training Accuracy: 99.09% 	 Test Accuracy: 98.90%
[ 4/10] 	 Time 74.21s 	 Training Accuracy: 99.35% 	 Test Accuracy: 99.00%
[ 5/10] 	 Time 71.32s 	 Training Accuracy: 99.50% 	 Test Accuracy: 99.05%
[ 6/10] 	 Time 72.00s 	 Training Accuracy: 99.52% 	 Test Accuracy: 99.13%
[ 7/10] 	 Time 72.08s 	 Training Accuracy: 99.48% 	 Test Accuracy: 98.85%
[ 8/10] 	 Time 74.15s 	 Training Accuracy: 99.71% 	 Test Accuracy: 99.10%
[ 9/10] 	 Time 73.69s 	 Training Accuracy: 99.81% 	 Test Accuracy: 99.12%
[10/10] 	 Time 66.82s 	 Training Accuracy: 99.72% 	 Test Accuracy: 99.12%


(99.71666666666667, 99.11666666666666)

In [34]:
f(x,y) = 2*x + 3*y

f (generic function with 1 method)

In [36]:
@code_llvm f(2,3)

;  @ In[34]:1 within `f`
define i64 @julia_f_23589(i64 signext %0, i64 signext %1) #0 {
top:
; ┌ @ int.jl:88 within `*`
   %2 = shl i64 %0, 1
   %3 = mul i64 %1, 3
; └
; ┌ @ int.jl:87 within `+`
   %4 = add i64 %3, %2
   ret i64 %4
; └
}
